In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_width, img_height = 150, 150
batch_size = 32
epochs = 5


train_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/train'
test_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/test'

In [ ]:
from keras.applications import vgg16

#base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),include_top=False,weights='imagenet')

base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
def addTopOnNet(bottom_model, num_classes):
    top_model = bottom_model.get_layer("block3_pool").output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(32,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(16,activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

In [ ]:
FC_Head = addTopOnNet(base_model, 1)

model = Model(inputs = base_model.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0   

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3951 images belonging to 2 classes.
Found 997 images belonging to 2 classes.


In [ ]:
nb_train_samples = 3951
nb_test_samples = 997

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 622s 5s/step - loss: 0.9736 - accuracy: 0.5359 - precision: 0.5320 - recall: 0.6093 - val_loss: 0.6504 - val_accuracy: 0.9567 - val_precision: 0.9236 - val_recall: 0.9960
Epoch 2/5
123/123 [==============================] - 24s 194ms/step - loss: 0.6381 - accuracy: 0.6226 - precision: 0.6092 - recall: 0.6805 - val_loss: 0.6105 - val_accuracy: 0.9859 - val_precision: 0.9728 - val_recall: 1.0000
Epoch 3/5
123/123 [==============================] - 24s 194ms/step - loss: 0.5290 - accuracy: 0.7364 - precision: 0.7223 - recall: 0.7705 - val_loss: 0.4744 - val_accuracy: 0.9738 - val_precision: 0.9538 - val_recall: 0.9960
Epoch 4/5
123/123 [==============================] - 24s 194ms/step - loss: 0.4362 - accuracy: 0.8147 - precision: 0.8341 - recall: 0.7856 - val_loss: 0.3501 - val_accuracy: 0.9496 - val_precision: 0.9870 - val_recall: 0.9116
Epoch 5/5
123/123 [==============================] - 24s 195ms/step - loss: 0.3937 - accuracy: 0.8

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 24s 196ms/step - loss: 0.3500 - accuracy: 0.8617 - precision: 0.8985 - recall: 0.8161 - val_loss: 0.1807 - val_accuracy: 0.9688 - val_precision: 0.9532 - val_recall: 0.9859
Epoch 2/5
123/123 [==============================] - 24s 194ms/step - loss: 0.2787 - accuracy: 0.8977 - precision: 0.9387 - recall: 0.8510 - val_loss: 0.1761 - val_accuracy: 0.9667 - val_precision: 0.9444 - val_recall: 0.9920
Epoch 3/5
123/123 [==============================] - 24s 194ms/step - loss: 0.3114 - accuracy: 0.8602 - precision: 0.9385 - recall: 0.7708 - val_loss: 0.2086 - val_accuracy: 0.9587 - val_precision: 0.9526 - val_recall: 0.9659
Epoch 4/5
123/123 [==============================] - 24s 195ms/step - loss: 0.2958 - accuracy: 0.8678 - precision: 0.9593 - recall: 0.7688 - val_loss: 0.2777 - val_accuracy: 0.9526 - val_precision: 0.9516 - val_recall: 0.9535
Epoch 5/5
123/123 [==============================] - 24s 194ms/step - loss: 0.2728 - accuracy: 0